In [2]:
from datasets import Dataset
from ragas.metrics import context_recall
from ragas import evaluate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

# 加载环境变量
load_dotenv()

# 从环境变量获取API密钥和基础URL
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_base = os.getenv("OPENAI_API_BASE")

# 创建LLM实例
llm = ChatOpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
    model="Pro/deepseek-ai/DeepSeek-V3"
)

# 创建Embedding模型实例
embeddings = OpenAIEmbeddings(
    api_key=openai_api_key,
    base_url=openai_api_base,
    model="BAAI/bge-m3"
)

# 准备数据样本
data_samples = {
    'user_input': ['埃菲尔铁塔位于哪里？'],
    'response': ['埃菲尔铁塔位于巴黎。'],
    'reference': ['埃菲尔铁塔位于巴黎。'],
    'retrieved_contexts': [
        ['巴黎是法国的首都。']
    ]
}

dataset = Dataset.from_dict(data_samples)

# 评估上下文召回率
score = evaluate(dataset, metrics=[context_recall], llm=llm, embeddings=embeddings)
result = score.to_pandas()
result

Evaluating: 100%|██████████| 1/1 [00:07<00:00,  7.94s/it]


,user_input,retrieved_contexts,response,reference,context_recall
0,埃菲尔铁塔位于哪里？,[巴黎是法国的首都。],埃菲尔铁塔位于巴黎。,埃菲尔铁塔位于巴黎。,1.0


In [3]:
from datasets import Dataset
from ragas.metrics import context_precision
from ragas import evaluate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from dotenv import load_dotenv
import os

# 加载环境变量与模型设置（与上例相同）
# ...

# 准备数据样本
data_samples = {
    'user_input': ['埃菲尔铁塔在哪里？'],
    'response': ['埃菲尔铁塔位于巴黎。'],
    'reference': ['埃菲尔铁塔位于巴黎，是法国的标志性建筑。'],
    'retrieved_contexts': [
        ['埃菲尔铁塔位于巴黎。']
    ]
}

dataset = Dataset.from_dict(data_samples)

# 评估上下文精确度
score = evaluate(dataset, metrics=[context_precision], llm=llm, embeddings=embeddings)
result = score.to_pandas()
result

Evaluating: 100%|██████████| 1/1 [00:11<00:00, 11.90s/it]


,user_input,retrieved_contexts,response,reference,context_precision
0,埃菲尔铁塔在哪里？,[埃菲尔铁塔位于巴黎。],埃菲尔铁塔位于巴黎。,埃菲尔铁塔位于巴黎，是法国的标志性建筑。,1.0


In [ ]:
from datasets import Dataset
from ragas.metrics import answer_correctness
from ragas import evaluate
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings  
from dotenv import load_dotenv
import os

# 加载环境变量与模型设置（与上例相同）
# ...

# 准备数据样本
data_samples = {
    'question': ['第一届超级碗是什么时候举行的？', '谁赢得的超级碗最多？'],
    'answer': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队赢得的超级碗最多'],
    'ground_truth': ['第一届超级碗于1967年1月15日举行', '新英格兰爱国者队创纪录地六次赢得超级碗']
}

dataset = Dataset.from_dict(data_samples)

# 评估答案正确性
score = evaluate(dataset, metrics=[answer_correctness], llm=llm, embeddings=embeddings)
result = score.to_pandas()
result

Evaluating: 100%|██████████| 2/2 [00:14<00:00,  7.06s/it]


,user_input,response,reference,answer_correctness
0,第一届超级碗是什么时候举行的？,第一届超级碗于1967年1月15日举行,第一届超级碗于1967年1月15日举行,1.000000
1,谁赢得的超级碗最多？,新英格兰爱国者队赢得的超级碗最多,新英格兰爱国者队创纪录地六次赢得超级碗,0.609829


In [ ]:
from transformers import BertTokenizer, BertModel
import torch
import numpy as np

# 加载预训练的 BERT 模型和分词器
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# 定义两个句子
sentence1 = "我喜欢看电影。"
sentence2 = "我很享受看电影。"

# 对句子进行编码
inputs1 = tokenizer(sentence1, return_tensors='pt', padding=True, truncation=True, max_length=128)
inputs2 = tokenizer(sentence2, return_tensors='pt', padding=True, truncation=True, max_length=128)

# 获取 BERT 输出
with torch.no_grad():
    outputs1 = model(**inputs1)
    outputs2 = model(**inputs2)

# 提取 [CLS] token 的表示（也可以用 mean pooling）
cls_embedding1 = outputs1.last_hidden_state[:, 0, :].squeeze().numpy()
cls_embedding2 = outputs2.last_hidden_state[:, 0, :].squeeze().numpy()

# 计算余弦相似度
cos_sim = np.dot(cls_embedding1, cls_embedding2) / (np.linalg.norm(cls_embedding1) * np.linalg.norm(cls_embedding2))
print(f"两个句子的余弦相似度: {cos_sim:.4f}")

两个句子的余弦相似度: 0.9979


In [4]:
# 构建更全面的评估数据集
eval_data = {
    'user_input': [
        '人工智能的三个主要分支是什么？',
        '什么是向量数据库？它在RAG系统中有什么作用？',
        '解释一下大语言模型的微调过程'
    ],
    'response': [
        '人工智能的三个主要分支是机器学习、深度学习和自然语言处理。',
        '向量数据库是专门存储和检索向量嵌入的数据库系统，在RAG系统中用于高效地检索语义相似的文档。',
        '大语言模型的微调是在预训练模型的基础上，使用特定任务的数据进一步训练模型的过程。'
    ],
    'reference': [
        '人工智能的三个主要分支是机器学习、自然语言处理和计算机视觉。',
        '向量数据库是专门设计用于存储和检索向量嵌入的数据库系统。在RAG系统中，它用于存储文档的向量表示，并通过相似性搜索快速检索与查询最相关的文档。',
        '大语言模型的微调是一个过程，其中预训练的基础模型使用特定领域或任务的数据进行额外训练，以优化其在特定应用场景的性能。'
    ],
    'retrieved_contexts': [
        ['人工智能主要包括机器学习、深度学习、自然语言处理和计算机视觉等领域。'],
        ['向量数据库是专门用于向量数据存储和检索的系统。在RAG应用中，向量数据库存储文档嵌入，通过计算相似度快速找到匹配查询的文档。'],
        ['大语言模型微调是在已预训练的模型基础上，使用特定数据集进行额外训练的过程，目的是使模型适应特定任务或领域。']
    ]
}

comprehensive_dataset = Dataset.from_dict(eval_data)

In [ ]:
from ragas.metrics import (
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity,
    answer_relevancy,
    faithfulness,
    
)

# 定义评估指标
metrics = [
    faithfulness,
    answer_relevancy,
    answer_similarity,
    answer_correctness,
    context_precision,
    context_recall,
]

# 综合评估
comprehensive_score = evaluate(
    comprehensive_dataset,
    metrics=metrics,
    llm=llm,
    embeddings=embeddings
)

# 结果分析
result_df = comprehensive_score.to_pandas()
result_df


ImportError: cannot import name 'context_relevancy' from 'ragas.metrics' (c:\Users\k\.conda\envs\XiaokeAILabs\lib\site-packages\ragas\metrics\__init__.py)